In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import random
import warnings
warnings.filterwarnings('ignore')
from time import gmtime, strftime

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import random

# Importing Datasets

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Data Exploration

In [3]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
train_df.target.value_counts(normalize=True)
# 10% of targets are 1, 90% are 0

0    0.89951
1    0.10049
Name: target, dtype: float64

In [5]:
test_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [6]:
train_df.dtypes

ID_code     object
target       int64
var_0      float64
var_1      float64
var_2      float64
var_3      float64
var_4      float64
var_5      float64
var_6      float64
var_7      float64
var_8      float64
var_9      float64
var_10     float64
var_11     float64
var_12     float64
var_13     float64
var_14     float64
var_15     float64
var_16     float64
var_17     float64
var_18     float64
var_19     float64
var_20     float64
var_21     float64
var_22     float64
var_23     float64
var_24     float64
var_25     float64
var_26     float64
var_27     float64
            ...   
var_170    float64
var_171    float64
var_172    float64
var_173    float64
var_174    float64
var_175    float64
var_176    float64
var_177    float64
var_178    float64
var_179    float64
var_180    float64
var_181    float64
var_182    float64
var_183    float64
var_184    float64
var_185    float64
var_186    float64
var_187    float64
var_188    float64
var_189    float64
var_190    float64
var_191    f

In [7]:
train_df.isnull().sum().sort_values(ascending=False)[:10]

var_199    0
var_61     0
var_71     0
var_70     0
var_69     0
var_68     0
var_67     0
var_66     0
var_65     0
var_64     0
dtype: int64

# Train and Test Dataset Have the Same Columns?

In [8]:
# Drop Different Columns from train and test
print('\nTrain and Test Datasets have the same columns?:',
      train_df.drop('target',axis=1).columns.tolist()==test_df.columns.tolist())
print("\nVariables not in test but in train : ", 
      set(train_df.drop('target',axis=1).columns).difference(set(test_df.columns)))
dif = list(set(train_df.drop('target',axis=1).columns).difference(set(test_df.columns)))


Train and Test Datasets have the same columns?: True

Variables not in test but in train :  set()


# Feature Engineering

### Merge train_df and test_df

In [9]:
y_train = train_df["target"].values
train_id = train_df['ID_code'].values
test_id = test_df['ID_code'].values

df_merge = pd.concat([train_df.drop('target',axis=1),test_df],axis=0)
df_merge.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [10]:
df_merge['sum'] = df_merge.drop(['ID_code'], axis=1).sum(axis=1).values
df_merge['mean'] = df_merge.drop(['ID_code'], axis=1).mean(axis=1).values
df_merge['std'] = df_merge.drop(['ID_code'], axis=1).std(axis=1).values
df_merge['min'] = df_merge.drop(['ID_code'], axis=1).min(axis=1).values
df_merge['max'] = df_merge.drop(['ID_code'], axis=1).max(axis=1).values
df_merge['skew'] = df_merge.drop(['ID_code'], axis=1).skew(axis=1).values
df_merge['kurt'] = df_merge.drop(['ID_code'], axis=1).kurtosis(axis=1).values
df_merge['med'] = df_merge.drop(['ID_code'], axis=1).median(axis=1).values

### Restore datraframes df_train and df_test

In [11]:
train_df = df_merge[:len(train_df)]
train_df["target"] = y_train.tolist()
test_df = df_merge[len(train_df):]

train_df['ID_code'] = train_id
test_df['ID_code'] = test_id

# LightGBModel

### GridSearchCV

In [12]:
# GridSearchCV to find best params for LightGBM

# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import StratifiedKFold
# params = {"objective" : ["binary"],
#           "metric" : ["auc"],
#           "num_leaves" : [3,9,15,60],
#           'num_iterations' : [5000],
#           'seed': [42],
#           "learning_rate" : [0.05],
#           "bagging_fraction" : [0.9],
#           "feature_fraction" : [0.3],
#           "bagging_seed" : [0]}

# lgb_model = lgb.LGBMRegressor(verbose=200)

# clf = GridSearchCV(estimator=lgb_model,
#                    param_grid=params,
#                    cv=StratifiedKFold(n_splits=3, shuffle=True), 
#                    refit=True, 
#                    verbose=True)

# X = train_df.drop(['ID_code', 'target'], axis=1)
# X_test = test_df.drop(['ID_code'], axis=1)
# y = train_df.target
# clf.fit(X, y)

In [13]:
X = train_df.drop(['ID_code', 'target'], axis=1)
# X_test = pca.transform(test_df.drop(['ID_code'], axis=1).values)
X_test = test_df.drop(['ID_code'], axis=1)
y = train_df.target

n_splits = 5
VERBOSE_EVAL = 300
STOP_ROUNDS = 100
print("lgb_model ...")

prediction = np.zeros(test_df.shape[0])

# folds = KFold(n_splits=n_splits, shuffle=True, random_state=42)
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold_n, (train_index, test_index) in enumerate(folds.split(X,y)):
    NUM_ROUNDS = int(random.uniform(15, 75)) # Number of boosting trees
    params = {"objective" : "binary",
          "metric" : "auc",
          "num_leaves" : int(random.uniform(2, 31)),
          'min_data_in_leaf': int(random.uniform(10, 20)),
          'num_iterations' : 999999, 
          'seed': 42,
          "learning_rate" : 0.05,
          "bagging_fraction" : random.uniform(0.3, 1),
          "feature_fraction" : random.uniform(0.3, 1),
          "bagging_seed" : 0,
          "num_threads": 4}
    lgb_model = lgb.LGBMRegressor(**params, n_estimators = NUM_ROUNDS, nthread = 2, n_jobs = -1)
    print('\n')
    print(lgb_model)
    
    print('Fold:', fold_n)
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    
    lgb_model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='auc',
            verbose=VERBOSE_EVAL, early_stopping_rounds=STOP_ROUNDS)
    
    y_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration_)
    prediction += y_pred
prediction /= n_splits

lgb_model ...


LGBMRegressor(bagging_fraction=0.47842391064598466, bagging_seed=0,
       boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       feature_fraction=0.814688984534055, importance_type='split',
       learning_rate=0.05, max_depth=-1, metric='auc',
       min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=13,
       min_split_gain=0.0, n_estimators=46, n_jobs=-1, nthread=2,
       num_iterations=999999, num_leaves=4, num_threads=4,
       objective='binary', random_state=None, reg_alpha=0.0,
       reg_lambda=0.0, seed=42, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)
Fold: 0
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.863429	valid_1's auc: 0.850387
[600]	training's auc: 0.890313	valid_1's auc: 0.873422
[900]	training's auc: 0.902578	valid_1's auc: 0.88288
[1200]	training's auc: 0.910027	valid_1's auc: 0.888219
[1500]	training's auc: 0.915042	valid_1's auc: 0.891576
[1800

# Submission

In [14]:
# y_pred = lgb_model.predict(pca.transform(test_df.drop(['ID_code'], axis=1).values))

In [15]:
# # Vector of probabilities
# y_pred

In [16]:
# y_train_lgb = lgb_model.predict(pca.transform(train_df.drop(['ID_code', 'target'], axis=1)))

In [17]:
# # Vector of probabilities
# y_train_lgb

### Finding Optimal Threshold

In [18]:
# def set_threshhold(y_train, y_train_model, threshhold=0.5):
#     y_train = y_train.copy()
#     y_train_model = y_train_model.copy()
#     y_train_model[y_train_model<threshhold] = 0
#     y_train_model[y_train_model>=threshhold] = 1
# #     print('Accuracy Score:', accuracy_score(y_train, y_train_model))
#     return accuracy_score(y_train, y_train_model)

# steps = np.linspace(0,1,1000)
# threshholds = []
# df_threshold = pd.DataFrame() #creates a new dataframe that's empty
# for step_index in range(len(steps)):
#     df_threshold.loc[step_index, 'StepValue'] = steps[step_index]
#     df_threshold.loc[step_index, 'AccuracyScore'] = set_threshhold(y_train=train_df.target,
#                                                                    y_train_model=y_train_lgb,
#                                                                    threshhold=steps[step_index])
    
# max_index = int(df_threshold[df_threshold.AccuracyScore == df_threshold.AccuracyScore.max()].index[0])

In [19]:
# # from sklearn.metrics import accuracy_score
# threshold_otimo = df_threshold.StepValue.iloc[max_index]
# y_train_lgb[y_train_lgb<threshold_otimo] = 0
# y_train_lgb[y_train_lgb>=threshold_otimo] = 1
# print('Optimal Accuracy Score:', accuracy_score(train_df.target.values, y_train_lgb))

In [20]:
# from sklearn.metrics import classification_report
# print(classification_report(train_df.target, y_train_lgb))

### Submit answer

In [21]:
# y_pred[y_pred<threshold_otimo] = 0
# y_pred[y_pred>=threshold_otimo] = 1

In [22]:
y_pred = prediction
test_df['target'] = y_pred

submission_string = 'lgb_' + str(n_splits) + 'splits' + strftime("%Y-%m-%d %H:%M:%S", gmtime()) + '.csv'
test_df.loc[:, ['ID_code', 'target']].to_csv(submission_string, index=False)